# Iterative Quantum Phase Estimation (IQPE)

This Notebook explains the use of the class tha implements **IQPE** using myqlm

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic
%matplotlib inline

Following cell instantiate the solver (QLM CESGA's qpu or simpl python simulator)

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg() 

In [ ]:
def getstaff(InputPDF):
    pdf = InputPDF.copy(deep=True)
    #Con esto obtenemos el theta del operador unitario
    pdf['Theta_Unitary'] = 2*np.pi*pdf['Phi']
    #Esto es el ángulo que rota el operador Q en el problema del valor esperador de la función (2*theta)
    #IQAE.results['Theta_Q'] = np.pi*IQAE.results['Phi']
    pdf['Theta'] = np.pi*pdf['Phi']
    #Solo angulos entre [0, pi/2]
    #IQAE.results['Theta'].where(IQAE.results['Theta']< 0.5*np.pi, np.pi-IQAE.results['Theta'], inplace=True)
    pdf['E_p(f)'] = np.sin(pdf['Theta'])**2
    pdf['theta_90'] = pdf['Theta']
    pdf['theta_90'].where(pdf['theta_90']< 0.5*np.pi, np.pi-pdf['theta_90'], inplace=True)    
    return pdf

## 1. Easy Test

First we are going to test the implemented class for **IQPE** using an easy example taking from qiskit textbook:

https://qiskit.org/textbook/ch-labs/Lab04_IterativePhaseEstimation.html

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb


In [ ]:
#Number Of Qbits
n_qbits = 2
#Number Of Classical Bits
n_cbits = 6

In [ ]:
#Basic Initial circuit and unitary operator whose autovalue we want to compute

from qat.lang.AQASM import Program, H, X, PH
q_prog = Program()
q_bits = q_prog.qalloc(n_qbits)
for i in range(n_qbits):
    q_prog.apply(X, q_bits[i])
q_gate = PH(np.pi/2.0).ctrl()

In [ ]:
init_circuit = q_prog.to_circ()

print('Initial Circuit')
%qatdisplay init_circuit

print('Unitary Operator')
%qatdisplay q_gate

Next We need to instantiate and configure the **IQPE** class

In [ ]:
from iterative_quantum_pe import IterativeQuantumPE

In [ ]:
iqpe_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : n_cbits,
    'shots': 1000,
    'easy': True
    #'easy': False
}
IQPE = IterativeQuantumPE(q_prog, q_gate, **iqpe_dict)
IQPE.iqpe()

In [ ]:
easy_circuit =IQPE.circuit
%qatdisplay easy_circuit  --depth 1  

In [ ]:
IQPE.results['Phi'].describe()

As can be seen in 

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb 

solution in qiskit is just 0.25 for the before configuration

## 2. Class IQPE: step by step 

We want to use IQPE algorithm to calculate the expected value of a function $f(x)$ for a x following a probability distribution $p$: $E_{x\sim p}(f)$. In this case we need the gates for loading probability ($\mathcal{P}$) and function ($\mathcal{R}$) and the corresponding Groover operator based on this gates ($\mathcal{Q}$). This is done in following cells

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
from QuantumMultiplexors_Module import expectation_loading_data
from PhaseAmplification_Module import load_q_gate

In [ ]:
#Functions f and p
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 6
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)
#Load P and R Gates
PR_Gate = expectation_loading_data(p_X, f_X)
#Creates Groover operator
Q_Gate = load_q_gate(PR_Gate)

In [ ]:
from qat.lang.AQASM import Program

In [ ]:
Qprog = Program()
qbits = Qprog.qalloc(PR_Gate.arity)
Qprog.apply(PR_Gate, qbits)

In [ ]:
init_circ = Qprog.to_circ()
print('Initial Circuit')
%qatdisplay init_circ

print('Groover Operator')
%qatdisplay Q_Gate

In [ ]:
#Load Class
from iterative_quantum_pe import IterativeQuantumPE

In [ ]:
n_cbits = 6
#We create a python dictionary for configuration of class
iqpe_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : n_cbits,
    'easy': True,
    #'easy': False    
}

In [ ]:
#Instanciate the class
IQPE = IterativeQuantumPE(Qprog, Q_Gate, **iqpe_dict)

In [ ]:
#Initialize the quantum program
IQPE.init_iqpe()

In [ ]:
#Now we have the initial quantum program stored in the property q_prog
circuit = IQPE.q_prog.to_circ(submatrices_only=True)
%qatdisplay circuit

In [ ]:
#Execute IQPE algorithm
IQPE.apply_iqpe()

In [ ]:
#Class have a method to create a quantum circuit from quantum program
IQPE.get_circuit()
circuit = IQPE.circuit
%qatdisplay circuit

In [ ]:
#Class have a method to generate a job from the circuit created in the previous cell
IQPE.get_job()
print(IQPE.job)

In [ ]:
#There is a method for executing a job
IQPE.get_job_result()
#Property job_result stores the results of the execution of the job
print(IQPE.job_result)

In [ ]:
#Finally we want to get the results in a straigtoforward way: we use method get_classicalbits
IQPE.get_classicalbits()
#In property results we store the results as a pandas DataFrame
IQPE.results

In [ ]:
#In order to get useful information we can use getstaff funciton.
#It claculates the angles and the expected value 
getstaff(IQPE.results)

## 3. Class IQPE: complete execution

In [ ]:
#Class IQPE have a method to do the complete workflow explained in section 2: iqpe

In [ ]:
#We create a python dictionary for configuration of class
n_cbits = 6
#We create a python dictionary for configuration of class
iqpe_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : n_cbits,
    #'easy': True,
    'easy': False    
}

In [ ]:
#Create the object
IQPE = IterativeQuantumPE(Qprog, Q_Gate, **iqpe_dict)
#Execute complete algorithm
IQPE.iqpe()

In [ ]:
IQPE.results

In [ ]:
getstaff(IQPE.results)

In [ ]:
#For comparing with E_p(f) column
sum(f_X*p_X)

In [ ]:
cir = IQPE.circuit
%qatdisplay cir

### Multiple executions

In [ ]:
#We can do several circuit executions configuring input dictionary properly
iqpe_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : n_cbits,
    'easy': False,
    'shots': 100
}
IQPE = IterativeQuantumPE(Qprog, Q_Gate, **iqpe_dict)
IQPE.iqpe()

In [ ]:
pdf_Results = getstaff(IQPE.results)

In [ ]:
pdf_Results.head()

In [ ]:
plt.hist(pdf_Results['E_p(f)'])

In [ ]:
pdf_Results.sort_values('Theta', inplace=True)
plt.plot(pdf_Results['Theta'], pdf_Results['Probability'], 'o-')

In [ ]:
pdf_Results.sort_values('theta_90', inplace=True)
plt.plot(pdf_Results['theta_90'], pdf_Results['Probability'], 'o-')

In [ ]:
plt.hist(pdf_Results['theta_90'])

In [ ]:
plt.hist(pdf_Results['Theta'])

In [ ]:
pdf_Results['E_p(f)'].describe()

In [ ]:
sum(p_X*f_X)